In [4]:
#| default_exp labeling

# AI-Assisted Data Analysis Notebook
This notebook is designed for AI-assisted data labeling and querying using OpenAI's APIs. It demonstrates modular programming practices, AI-assisted labeling, and dataset querying functionalities. 

In [6]:
#| export
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import pandas as pd
from sklearn.metrics import accuracy_score
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
import os
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'


## Function `train_model`
This function trains a model on labeled data. It filters out unlabeled data, encodes labels, loads a pre-trained model and tokenizer, tokenizes input data, creates a training dataset, and trains the model.



## Function `evaluate_model_accuracy`
This function (not fully visible in the provided snippet) is designed to evaluate the accuracy of the trained model, likely using the `accuracy_score` function to compare predicted labels against the true labels.


## Function `iterative_training`
Implements an iterative training process to refine the model with user feedback. In each iteration, it retrains the model, evaluates its accuracy, and incorporates user feedback for model refinement.


## Function `predict_and_update_labels`
Uses the trained model to predict labels for unlabeled data in the DataFrame. It iterates over the DataFrame, predicts labels for rows marked as 'UNLABELED', and updates the DataFrame with these labels.




In [7]:
#| export
def train_model(df, model_name='bert-base-uncased', num_labels=4):
    """Train a model on the labeled data."""
    # Filter out unlabeled data
    labeled_df = df[df['Label'] != 'UNLABELED']

    # Encode labels as integers
    label_encoder = LabelEncoder()
    labeled_df['EncodedLabel'] = label_encoder.fit_transform(labeled_df['Label'])

    # Load a pre-trained model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Tokenize the input data
    tokenized_data = tokenizer(labeled_df['Text'].tolist(), padding=True, truncation=True, max_length=512)

    # Create a dataset for training
    train_dataset = Dataset.from_dict({**tokenized_data, 'labels': labeled_df['EncodedLabel'].tolist()})

    # Define training arguments
    training_args = TrainingArguments(output_dir='./results', num_train_epochs=3, per_device_train_batch_size=16)

    # Initialize the Trainer
    trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset)

    # Move model to CPU
    model.to('cpu')

    # Train the model
    trainer.train()

    return model, tokenizer

def evaluate_model_accuracy(df, model, tokenizer):
    """Evaluate the model accuracy based on user labels."""
    labeled_df = df[df['Label'] != 'UNLABELED']

    # Prepare the data for the model
    inputs = tokenizer(labeled_df['Text'].tolist(), padding=True, truncation=True, return_tensors="pt")

    # Move inputs to CPU
    inputs = {k: v.to('cpu') for k, v in inputs.items()}

    # Make sure the model is on CPU
    model.to('cpu')

    # Get predictions
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(dim=1)

    # Convert predictions to label names
    label_map = {0: 'OTR', 1: 'PRS', 2: 'REP', 3: 'NEU'}
    predicted_labels = [label_map[label] for label in predictions.numpy()]

    # Calculate accuracy
    accuracy = accuracy_score(labeled_df['Label'].tolist(), predicted_labels)

    return accuracy

def iterative_training(df, model, tokenizer, max_iterations=10):
    """Train the model iteratively, improving with user feedback."""
    for iteration in range(max_iterations):
        model, tokenizer = train_model(df)

        # Evaluate accuracy
        accuracy = evaluate_model_accuracy(df, model, tokenizer)
        print(f"Iteration {iteration+1}: Model Accuracy = {accuracy}")

        # Integrate with UI to get updated labels from the user
        # df = get_updated_labels_from_ui()  # This needs to be implemented in the Streamlit script

        # Check for stopping condition
        # if stopping_condition:
        #     break

    return model, tokenizer

# You might need to define additional helper functions or import additional libraries depending on your requirements.


from transformers import pipeline

def predict_and_update_labels(df, model, tokenizer):
    """Use the trained model to predict labels for unlabeled data and update the DataFrame."""
    # Creating a pipeline for text classification
    text_classification_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=False)

    # Define the label mapping based on your model's training
    label_map = {0: 'OTR', 1: 'PRS', 2: 'REP', 3: 'NEU'}

    # Iterating over the DataFrame and predicting labels for unlabeled rows
    for i, row in df.iterrows():
        if row['Label'] == 'UNLABELED':
            # Make prediction
            prediction = text_classification_pipeline(row['Text'])
            predicted_label_index = int(prediction[0]['label'].split('_')[-1])  # Extract index from 'LABEL_X'
            # Update the DataFrame with the predicted label name
            df.at[i, 'Label'] = label_map[predicted_label_index]

    return df

## Example Usage
Demonstration of how this notebook can be used to iteratively train a model to label data and refine it with user feedback.
```python
# Load dataset
df = pd.read_csv('data.csv')

# Initial training and labeling
model, tokenizer = train_model(df)
df = predict_and_update_labels(df, model, tokenizer)

# Iterative training with user feedback
for iteration in range(3):  # Example of 3 iterations
    model, tokenizer = iterative_training(df, model, tokenizer)
    df = predict_and_update_labels(df, model, tokenizer)
    # Incorporate user feedback here

# Evaluate final model accuracy
accuracy = evaluate_model_accuracy(df, model, tokenizer)
print(f"Final Model Accuracy: {accuracy}")

# Save the labeled data
df.to_csv('labeled_data.csv')